# Cariberation (TOP LEFT -> Bottom Right)

In [1]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands


# text cv2 puttext

font                   = cv2.FONT_HERSHEY_SIMPLEX
location = (100,50)
fontScale              = 1
fontColor              = (255,255,255)
lineType               = 2
cali=[]

# For webcam input:
hands = mp_hands.Hands(
    min_detection_confidence=0.75, min_tracking_confidence=0.75)
cap = cv2.VideoCapture(0)
countdown=100
while cap.isOpened():
    success, image = cap.read()
    if not success:
        break

  # Flip the image horizontally for a later selfie-view display, and convert
  # the BGR image to RGB.
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
  # To improve performance, optionally mark the image as not writeable to
  # pass by reference.
    image.flags.writeable = False
    results = hands.process(image)

  # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
        cv2.putText(image,str(countdown/5), location, font, fontScale,fontColor,lineType)
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            if countdown==0:
                land = hand_landmarks
                val = land.landmark
                cali.append([val[8].x,val[8].y])
                countdown=100
    else:
        cv2.putText(image,'No Hands!', location, font, fontScale,fontColor,lineType)
    cv2.imshow('MediaPipe Hands', image)
    key = cv2.waitKey(1)
    countdown-=1
    if key == 27:
        break
    if len(cali)==2:
        break
hands.close()
cap.release()
cv2.destroyAllWindows()

# Calculations

In [2]:
print(cali)
from sympy import symbols, Eq, solve 

x, y = symbols('x,y') 
  
# defining equations 
eq1 = Eq((x*cali[0][0]+y), 0) 
eq2 = Eq((x*cali[1][0]+y), 1920) 
  
e1=solve((eq1, eq2), (x, y))
l1=list(e1.values())


x, y = symbols('x,y') 
  
# defining equations 
eq1 = Eq((x*cali[0][1]+y), 0) 
eq2 = Eq((x*cali[1][1]+y), 1080) 
e2=solve((eq1, eq2), (x, y))
l2=list(e2.values())


print(l1)
print(l2)

[[0.4845822751522064, 0.16618086397647858], [0.7856217622756958, 0.6307530999183655]]
[6377.90084731440, -3090.61770328680]
[2324.71920714414, -386.323846345928]


# Cursur Movement (No click)

In [16]:
import pyautogui
pyautogui.FAILSAFE= False
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands


# text cv2 puttext

font                   = cv2.FONT_HERSHEY_SIMPLEX
location = (100,50)
fontScale              = 1
fontColor              = (255,255,255)
lineType               = 2


# For webcam input:
hands = mp_hands.Hands(
    min_detection_confidence=0.75, min_tracking_confidence=0.75)
cap = cv2.VideoCapture(0)
while cap.isOpened():
    success, image = cap.read()
    if not success:
        break

  # Flip the image horizontally for a later selfie-view display, and convert
  # the BGR image to RGB.
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
  # To improve performance, optionally mark the image as not writeable to
  # pass by reference.
    image.flags.writeable = False
    results = hands.process(image)

  # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            land = hand_landmarks
            val = land.landmark
            x=val[8].x*l1[0]+l1[1]
            if(x<0):
                x=0
            if(x>1920):
                x=1920
            y=val[8].y*l2[0]+l2[1]
            if(y<0):
                y=0
            if(y>1080):
                y=1080
            pyautogui.moveTo(x, y)
    else:
        cv2.putText(image,'No Hands!', location, font, fontScale,fontColor,lineType)
    cv2.imshow('MediaPipe Hands', image)
    key = cv2.waitKey(1)
    if key == 27:
        break
hands.close()
cap.release()
cv2.destroyAllWindows()

In [4]:
import pyautogui
print(pyautogui.size())

Size(width=1920, height=1080)


# Mouse

In [3]:
from keras.models import load_model
import numpy as np
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
model=load_model("curmodel")
import pyautogui
pyautogui.FAILSAFE= False

Using TensorFlow backend.


In [4]:
Char="0123456789DOU$"

font        = cv2.FONT_HERSHEY_SIMPLEX
location    = (100,50)
fontScale   = 1
fontColor   = (255,255,255)
lineType    = 2


hands = mp_hands.Hands(
    min_detection_confidence=0.5, min_tracking_confidence=0.5)
cap = cv2.VideoCapture(0)
ccount=20
while cap.isOpened():
    success, image = cap.read()
    if not success:
        break

    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = hands.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
        L=[]
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            land = hand_landmarks
            val = land.landmark
            first=val[0];
            for j in val:
                L.append(j.x-first.x);
                L.append(j.y-first.y);
                L.append(j.z-first.z);
            break
        L=L[3::]
        L=np.array(L)
        x = np.expand_dims(L,0)
        x = np.expand_dims(x,-1)
        fi=int(np.argmax(model.predict(x)))
        if fi==1:
                for hand_landmarks in results.multi_hand_landmarks:
                    land = hand_landmarks
                    val = land.landmark
                    x=val[8].x*l1[0]+l1[1]
                    if(x<0):
                        x=0
                    if(x>1920):
                        x=1920
                    y=val[8].y*l2[0]+l2[1]
                    if(y<0):
                        y=0
                    if(y>1080):
                        y=1080
                    pyautogui.moveTo(x, y)
        if fi==6:
            ccount-=1
            for hand_landmarks in results.multi_hand_landmarks:
                land = hand_landmarks
                val = land.landmark
                x=val[8].x*l1[0]+l1[1]
                if(x<0):
                    x=0
                if(x>1920):
                    x=1920
                y=val[8].y*l2[0]+l2[1]
                if(y<0):
                    y=0
                if(y>1080):
                    y=1080
                if(ccount==0):
                    ccount=20
                    print("Single click")
                    pyautogui.click(x, y)
        if fi==2:
            ccount-=1
            for hand_landmarks in results.multi_hand_landmarks:
                land = hand_landmarks
                val = land.landmark
                x=val[8].x*l1[0]+l1[1]
                if(x<0):
                    x=0
                if(x>1920):
                    x=1920
                y=val[8].y*l2[0]+l2[1]
                if(y<0):
                    y=0
                if(y>1080):
                    y=1080
                if(ccount==0):
                    ccount=20
                    print("Double clicked")
                    pyautogui.click(x, y,clicks=2)
    else:
        cv2.putText(image,'No Hands!', location, font, fontScale,fontColor,lineType)
    
    cv2.imshow('MediaPipe Hands', image)
    key = cv2.waitKey(1)
    if key == 27:
        break
hands.close()
cap.release()
cv2.destroyAllWindows()

Single click
Single click
Single click
Single click
Single click
Single click
